<a href="https://colab.research.google.com/github/HungNguyen205/HK5--ThucHanhTriTueNhanTao/blob/main/THTTNT_2001230284_NguyenCaoManhHung_Buoi04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align ="center"><font color ="purple">⛅Báo Cáo Thực Hành Trí Tuệ Nhân Tạo</font></h1>
<h3>🚩Tuần 4 - Chủ đề: CÁC PHƯƠNG PHÁP GIẢI BÀI TOÁN THỎA MÃN RÀNG BUỘC</h3>

---

**Họ và tên:** Nguyễn Cao Mạnh Hùng<br/>
**MSSV:** 2001230284
